# Function Definition and initialization

In [ ]:
import hashlib
import os
import requests
import json
import sys
from openai import AzureOpenAI

from config.azure_openai_config import API_KEY, AZURE_ENDPOINT

In [ ]:
def generate_image(prompt, show_image=False, quality=None ,imageCount=1, size=None):
    client = AzureOpenAI(
        api_version="2024-02-01",
        api_key=API_KEY,  
        azure_endpoint=AZURE_ENDPOINT,
    )

    if imageCount not in [1, 2, 4]:
        print("imageCount should be 1, 2, or 4")
        imageCount = 1
    if size is not None:
        sizeS = size
    try :
        result = client.images.generate(
            model="dalle3",
            prompt=prompt,
            n=imageCount,
            quality=quality,
            size=sizeS,
        )
    except Exception as e:
        print(f"Error generating image: {e}")
        return None
    
    json_response = json.loads(result.model_dump_json())

    image_filenames = []
    for i in range(imageCount):
        image_url = json_response["data"][i]["url"]  # extract image URL from response
        generated_image = requests.get(image_url).content  # download the image

        checksum = hashlib.md5(generated_image).hexdigest()
        image_filename = f"{checksum}.png"

        image_dir = os.path.join(os.curdir, 'images')
        if not os.path.isdir(image_dir):
            os.mkdir(image_dir)
        
        image_path = os.path.join(image_dir, image_filename)

        with open(image_path, "wb") as image_file:
            image_file.write(generated_image)

        image_filenames.append(image_filename)

    if imageCount == 1:
        return image_filename
    else:
        return image_filenames

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

def show_image(image_path):
    try:
        image = Image.open(image_path)
        plt.imshow(image)
        plt.axis('off')  # 不顯示坐標軸
        plt.show()
    except Exception as e:
        print(f"無法顯示圖像: {e}")

In [ ]:
prompt = "photorealistic, if there is no love, what then, surreal, finely detailed, explained"
filename = generate_image(prompt, size="1024x1024")
if filename:
    print(filename)
    show_image('images/' + filename)